# demo: 高雄市停車資訊

- 不是每一次資料都是沒問題的....

In [1]:
import requests

url = "https://data.kcg.gov.tw/dataset/449e45d9-dead-4873-95a9-cc34dabbb3af/resource/fe3f93da-9673-4f7b-859c-9017d793f798/download/108.6.21.csv"

r = requests.get(url)

print(r)

<Response [200]>


In [2]:
import csv

decoded_content = r.content.decode('utf-8')
cr = csv.reader(decoded_content.splitlines(), delimiter=',')
data_list = list(cr)


In [3]:
print(data_list[0])
print(data_list[1])

['\ufeff型式', '行政區', '場名', '位置', '緯度', '經度', '收費標準', '大車', '小車', '機車', '管理業者', '聯絡電話', '履約起迄']
['平面', '甲仙區', '甲仙林森', 'A區：林森路70號旁', '23.082862', '120.587988', '小車計次(30元/6小時)', '0', '35', '0', '歐特儀股份有限公司', '903135731', '113/04/01～119/03/31']


- 如果你不方便抓資料，或是資料連結已經遺失，可以直接讀檔案
- 這一段程式使用colab上傳csv檔
```python
#使用colab讀csv檔前，需要先使用這行來上傳檔案
from google.colab import files
uploaded = files.upload()
```

- 這一段程式是在個人電腦讀取csv檔

```python
import csv
file = open('kh_parking.csv', encoding="utf-8")
cr = csv.reader(file)
data_list = list(cr)
```

- or更好的寫法!

```python
with open('kh_parking.csv', 'r', encoding="utf-8") as f:
    cr = csv.reader(f)
    data_list = list(cr)
data_list[0:5]
```


### 開始畫圖

In [4]:
import folium
import geopy

from geopy.geocoders import Nominatim

def get_gps_coordinates(location_name):
    """
    取得指定地名的 GPS 座標。

    Args:
        location_name (str): 地名。

    Returns:
        tuple: (緯度, 經度)，如果找不到則返回 None。
    """
    try:
        geolocator = Nominatim(user_agent="my_geocoder")
        location = geolocator.geocode(location_name)
        if location:
            return location.latitude, location.longitude
        else:
            return None
    except Exception as e:
        print(f"發生錯誤：{e}")
        return None



In [5]:
location = get_gps_coordinates('高雄市')
print(location)

(22.6203348, 120.3120375)


In [8]:

m = folium.Map(location=location, zoom_start=15)

for item in data_list[1:]:
    try:
        name = item[2]  #站名
        lat = item[4]  #緯度
        lng = item[5]  #經度
        car_n = item[7]
        fee = item[9]
        info = '[%s] 小型車格數=%s, 收費=%s' %(name, car_n, fee)
        folium.Marker([ float(lat), float(lng)],
                      tooltip=info,
                     icon=folium.Icon(color='blue', prefix='fa', icon='fa-car')
                     ).add_to(m)
    except Exception as e:
        print(e.args)

In [9]:
m